In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [2]:
# Generate random dates

from datetime import date

# cannot use strftime()'s %B format since it depends on the locale
MONTHS = ["January", "February", "March", "April", "May", "June",
          "July", "August", "September", "October", "November", "December"]

def random_dates(n_dates):
    min_date = date(1000, 1, 1).toordinal()
    max_date = date(9999, 12, 31).toordinal()

    ordinals = np.random.randint(max_date - min_date, size=n_dates) + min_date
    dates = [date.fromordinal(ordinal) for ordinal in ordinals]

    x = [MONTHS[dt.month - 1] + " " + dt.strftime("%d, %Y") for dt in dates]
    y = [dt.isoformat() for dt in dates]
    return x, y

np.random.seed(42)

n_dates = 3
x_example, y_example = random_dates(n_dates)
print("{:25s}{:25s}".format("Input", "Target"))
print("-" * 50)
for idx in range(n_dates):
    print("{:25s}{:25s}".format(x_example[idx], y_example[idx]))

Input                    Target                   
--------------------------------------------------
September 20, 7075       7075-09-20               
May 15, 8579             8579-05-15               
January 11, 7103         7103-01-11               


In [6]:
INPUT_CHARS = "".join(sorted(set("".join(MONTHS) + "0123456789, ")))
OUTPUT_CHARS = "0123456789-"

def date_str_to_ids(date_str, chars=INPUT_CHARS):
    return [chars.index(c) for c in date_str]

def ids_to_date_strs(ids, chars=OUTPUT_CHARS):
    return ["".join([("?" + chars)[index] for index in sequence]) for sequence in ids]

def prepare_date_strs(date_strs, chars=INPUT_CHARS):
    X_ids = [date_str_to_ids(dt, chars) for dt in date_strs]
    X = tf.ragged.constant(X_ids, ragged_rank=1)
    return (X + 1).to_tensor() # using 0 as the padding token ID

def create_dataset(n_dates):
    x, y = random_dates(n_dates)
    return prepare_date_strs(x, INPUT_CHARS), prepare_date_strs(y, OUTPUT_CHARS)



In [26]:
np.random.seed(42)

X_train, Y_train = create_dataset(10000)
X_valid, Y_valid = create_dataset(2000)
X_test, Y_test = create_dataset(2000)

In [9]:
embedding_size = 32
max_output_length = Y_train.shape[1]

np.random.seed(42)
tf.random.set_seed(42)

encoder = tf.keras.Sequential([
    # map each character to an embedded vector of size 32
    tf.keras.layers.Embedding(input_dim=len(INPUT_CHARS) + 1, output_dim=embedding_size, input_shape=[None]),
    tf.keras.layers.LSTM(128)
])

decoder = tf.keras.Sequential([
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.Dense(len(OUTPUT_CHARS) + 1, activation="softmax")
])

model = tf.keras.Sequential([
    encoder,
    # the decoder expects a sequence, we simply repeat the encoder's output 
    tf.keras.layers.RepeatVector(max_output_length),
    decoder
])

optimizer = tf.keras.optimizers.Nadam()
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history = model.fit(X_train, Y_train, epochs=20, validation_data=(X_valid, Y_valid))

Epoch 1/20
313/313 [==============================] - 8s 18ms/step - loss: 1.6968 - accuracy: 0.3943 - val_loss: 1.2574 - val_accuracy: 0.5494
Epoch 2/20
313/313 [==============================] - 5s 16ms/step - loss: 1.2464 - accuracy: 0.5571 - val_loss: 0.9858 - val_accuracy: 0.6399
Epoch 3/20
313/313 [==============================] - 5s 17ms/step - loss: 0.9658 - accuracy: 0.6502 - val_loss: 0.8167 - val_accuracy: 0.6928
Epoch 4/20
313/313 [==============================] - 6s 18ms/step - loss: 0.7115 - accuracy: 0.7278 - val_loss: 0.6518 - val_accuracy: 0.7391
Epoch 5/20
313/313 [==============================] - 6s 18ms/step - loss: 0.5578 - accuracy: 0.7796 - val_loss: 0.4684 - val_accuracy: 0.8116
Epoch 6/20
313/313 [==============================] - 6s 19ms/step - loss: 0.6207 - accuracy: 0.7728 - val_loss: 0.8151 - val_accuracy: 0.7132
Epoch 7/20
313/313 [==============================] - 6s 19ms/step - loss: 0.4086 - accuracy: 0.8500 - val_loss: 0.2917 - val_accuracy: 0.8917

In [7]:
X_new = prepare_date_strs(["September 17, 2009", "July 14, 1789"])
ids = model.predict(X_new).argmax(axis=-1)
for date_str in ids_to_date_strs(ids):
    print(date_str)

1/1 [==============================] - 0s 424ms/step
2009-09-17
1789-07-14


In [10]:
# Version 2 - teacher forcing encoder-decoder

sos_id = len(OUTPUT_CHARS) + 1

def shifted_output_sequences(Y):
    sos_tokens = tf.fill(dims=(len(Y), 1), value=sos_id)
    return tf.concat([sos_tokens, Y[:, :-1]], axis=1)

X_train_decoder = shifted_output_sequences(Y_train)
X_valid_decoder = shifted_output_sequences(Y_valid)
X_test_decoder = shifted_output_sequences(Y_test)

In [11]:
encoder_embedding_size = 32
decoder_embedding_size = 32
lstm_units = 128

np.random.seed(42)
tf.random.set_seed(42)

# Encoder
encoder_input = tf.keras.layers.Input(shape=[None], dtype=tf.int32)
encoder_embedding = tf.keras.layers.Embedding(input_dim=len(INPUT_CHARS) + 1, output_dim=encoder_embedding_size)(encoder_input)
_, encoder_state_h, encoder_state_c = tf.keras.layers.LSTM(lstm_units, return_state=True)(encoder_embedding)
encoder_state = [encoder_state_h, encoder_state_c]

# Decoder
decoder_input = tf.keras.layers.Input(shape=[None], dtype=tf.int32)
decoder_embedding = tf.keras.layers.Embedding(input_dim=len(OUTPUT_CHARS) + 2, output_dim=decoder_embedding_size)(decoder_input)
decoder_lstm_output = tf.keras.layers.LSTM(lstm_units, return_sequences=True)(decoder_embedding, initial_state=encoder_state)
decoder_output = tf.keras.layers.Dense(len(OUTPUT_CHARS) + 1, activation="softmax")(decoder_lstm_output)

model = tf.keras.Model(inputs=[encoder_input, decoder_input], outputs=[decoder_output])

optimizer = tf.keras.optimizers.Nadam()
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history = model.fit([X_train, X_train_decoder], Y_train, epochs=10, validation_data=([X_valid, X_valid_decoder], Y_valid))

Epoch 1/10
313/313 [==============================] - 8s 19ms/step - loss: 1.6375 - accuracy: 0.3944 - val_loss: 1.3159 - val_accuracy: 0.5192
Epoch 2/10
313/313 [==============================] - 6s 19ms/step - loss: 1.0655 - accuracy: 0.6208 - val_loss: 0.8243 - val_accuracy: 0.7136
Epoch 3/10
313/313 [==============================] - 6s 20ms/step - loss: 0.6308 - accuracy: 0.7791 - val_loss: 0.4539 - val_accuracy: 0.8440
Epoch 4/10
313/313 [==============================] - 6s 20ms/step - loss: 0.2962 - accuracy: 0.9120 - val_loss: 0.1996 - val_accuracy: 0.9470
Epoch 5/10
313/313 [==============================] - 7s 21ms/step - loss: 0.1087 - accuracy: 0.9824 - val_loss: 0.0694 - val_accuracy: 0.9936
Epoch 6/10
313/313 [==============================] - 7s 23ms/step - loss: 0.0720 - accuracy: 0.9898 - val_loss: 0.0495 - val_accuracy: 0.9956
Epoch 7/10
313/313 [==============================] - 7s 24ms/step - loss: 0.0310 - accuracy: 0.9986 - val_loss: 0.0230 - val_accuracy: 0.9995

In [14]:
max_input_length = X_train.shape[1]

def prepare_date_strs_padded(date_strs):
    X = prepare_date_strs(date_strs)
    if X.shape[1] < max_input_length:
        X = tf.pad(X, [[0, 0], [0, max_input_length - X.shape[1]]])
    return X

def convert_date_strs(date_strs):
    X = prepare_date_strs_padded(date_strs)
    ids = model.predict(X).argmax(axis=-1)
    return ids_to_date_strs(ids)

sos_id = len(OUTPUT_CHARS) + 1

def predict_date_strs(date_strs):
    X = prepare_date_strs_padded(date_strs)
    Y_pred = tf.fill(dims=(len(X), 1), value=sos_id)

    for index in range(max_output_length):
        pad_size = max_output_length - Y_pred.shape[1]
        X_decoder = tf.pad(Y_pred, [[0, 0], [0, pad_size]])
        Y_probas_next = model.predict([X, X_decoder])[:, index:index+1]
        Y_pred_next = tf.argmax(Y_probas_next, axis=-1, output_type=tf.int32)
        Y_pred = tf.concat([Y_pred, Y_pred_next], axis=1)
        
    return ids_to_date_strs(Y_pred[:, 1:])

In [25]:
predict_date_strs(["May 02, 2020", "July 14, 1789", "Augussst 22, 1778"])

1/1 [==============================] - 0s 10ms/step


['2020-05-02', '1789-07-14', '1778-08-22']